In [1]:
import os

os.environ['CUDA_VISIBLE_DEVICES'] = ''

In [3]:
import malaya

model_en_ms = malaya.translation.en_ms.transformer(model = 'noisy-base')
model_ms_en = malaya.translation.ms_en.transformer(model = 'noisy-base')

2022-10-22 22:31:32.568674: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-10-22 22:31:32.572728: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2022-10-22 22:31:32.572748: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:169] retrieving CUDA diagnostic information for host: husein-MS-7D31
2022-10-22 22:31:32.572752: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:176] hostname: husein-MS-7D31
2022-10-22 22:31:32.572813: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:200] libcuda reported version is: Not found: was unable to find libcuda.so DSO loaded into this program
2022-10-22 22:31:32.572836: I tensorflow/stream

Downloading:   0%|          | 0.00/233M [00:00<?, ?B/s]

In [4]:
from sacrebleu.metrics import BLEU, CHRF, TER

bleu = BLEU()
chrf = CHRF(word_order = 2)

In [5]:
from unidecode import unidecode
import json

with open('test-noisy-shuffled.json') as fopen:
    test = fopen.read().split('\n')
    test = [json.loads(t) for t in test if len(t)]
    
len(test)

6854

In [7]:
test[0]['translation']

{'src': 'These scarps were found alll over tie moon and appear to be minimally weathered, indicating vat the geologic events hart creatd them wore fairly recent.',
 'tgt': 'Serpihan ini telah ditemui di serata permukaan bulan dan masih kurang terjejas oleh faktor persekitaran, yakni secara geologinya menunjukkan bahan tersebut adalah baharu terbentuk.',
 'prefix': 'terjemah Inggeris ke Melayu: '}

In [8]:
model_en_ms.greedy_decoder([test[0]['translation']['src']])

['Selendang ini ditemui di seluruh bulan dan kelihatan tidak lapuk secara minimum, menunjukkan bahawa peristiwa geologi yang mencipta mereka adalah agak baru-baru ini.']

In [13]:
from tqdm import tqdm

results_en_ms, filtered_right_en_ms = [], []
results_ms_en, filtered_right_ms_en = [], []
for i in tqdm(range(len(test))):
    t = test[i]['translation']
    p = t['prefix']
    s = t['src']
    tgt = t['tgt']
    
    if not len(s) or not len(tgt):
        continue
        
    if 'Inggeris ke Melayu' in p:
        o = model_en_ms.greedy_decoder([s])[0]
        results_en_ms.append(o)
        filtered_right_en_ms.append(tgt)
    else:
        o = model_ms_en.greedy_decoder([s])[0]
        results_ms_en.append(o)
        filtered_right_ms_en.append(tgt)

100%|███████████████████████████████████████| 6854/6854 [29:04<00:00,  3.93it/s]


In [14]:
len(results_en_ms), len(results_ms_en)

(2937, 3917)

In [15]:
refs = [filtered_right_en_ms]
sys = results_en_ms
r = bleu.corpus_score(sys, refs)
r.__dict__, chrf.corpus_score(sys, refs)

({'name': 'BLEU',
  'score': 41.8278308435666,
  '_mean': -1.0,
  '_ci': -1.0,
  '_verbose': '73.1/49.7/35.3/25.4 (BP = 0.985 ratio = 0.985 hyp_len = 63506 ref_len = 64473)',
  'bp': 0.9848884354831119,
  'counts': [46414, 30108, 20323, 13889],
  'totals': [63506, 60569, 57632, 54695],
  'sys_len': 63506,
  'ref_len': 64473,
  'precisions': [73.08600762132711,
   49.70859680694745,
   35.263395335924486,
   25.393546027973308],
  'prec_str': '73.1/49.7/35.3/25.4',
  'ratio': 0.9850014734850248},
 chrF2++ = 66.46)

In [16]:
refs = [filtered_right_ms_en]
sys = results_ms_en
r = bleu.corpus_score(sys, refs)
r.__dict__, chrf.corpus_score(sys, refs)

({'name': 'BLEU',
  'score': 40.77050068227297,
  '_mean': -1.0,
  '_ci': -1.0,
  '_verbose': '72.0/48.5/34.8/25.5 (BP = 0.972 ratio = 0.972 hyp_len = 90382 ref_len = 92985)',
  'bp': 0.9716107843891983,
  'counts': [65074, 41942, 28753, 20040],
  'totals': [90382, 86465, 82548, 78631],
  'sys_len': 90382,
  'ref_len': 92985,
  'precisions': [71.9988493284061,
   48.5074885791939,
   34.83185540533992,
   25.486131423993083],
  'prec_str': '72.0/48.5/34.8/25.5',
  'ratio': 0.9720062375651987},
 chrF2++ = 63.68)